In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import os

In [ ]:
def merge_vehicle_count(caracteristiques, vehicules):
    # Compter le nombre de véhicules par "Num_Acc" dans le DataFrame vehicules
    vehicules_count = vehicules.groupby("Accident_Id")["id_vehicule"].count().reset_index().rename(columns={'id_vehicule':'Nombre_vehicule'})
    
    # Fusionner le DataFrame caracteristiques avec le DataFrame vehicules_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(vehicules_count, on='Accident_Id')
    
    return merged_df

def merge_user_count(caracteristiques,usagers):
    # Compter le nombre d'usagers par "Accident_Id" et par sexe dans le DataFrame usagers
    usagers_count = usagers.groupby(["Accident_Id", "sexe"])["id_vehicule"].count().unstack(fill_value=0)
    usagers_count.rename(columns={1: 'Nombre_Hommes', 2: 'Nombre_Femmes'}, inplace=True)
    
    # Fusionner le DataFrame caracteristiques avec usagers_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(usagers_count, on='Accident_Id')
    
    return merged_df

def replace_with_mapper(df, col, mapper):
    df[col] = df[col].replace(mapper)
    return df

# Remplacement des heures par le moment de la journée
matinee = ['06:','07:','08:','09:','10:']
journee = ['11:','12:','13:','14:','15:','16:','17:']
soiree = ['18:','19:','20:','21:','22:','23:']
nuit = ['00:','01:','02:','03:','04:','05:']

def replace_hour(hour):
    hour = hour[:3]
    if hour in matinee:
        return 'Matinée'
    elif hour in journee:
        return 'Journée'
    elif hour in soiree:
        return 'Soirée'
    elif hour in nuit:
        return 'Nuit'
    else:
        'Lucas'

In [ ]:
years = ['19', '20', '21', '22']
files_name = ["caracteristiques-20", "lieux-20", "usagers-20", "vehicules-20"]
#os.mkdir('./Dataset/all_years/', 0o666) 
for file_name in files_name:
    result = []
    for year in years:
        file_path = f'./Dataset/{year}/{file_name}{year}.csv'
        sep = ';'
        df = pd.read_csv(file_path, sep=sep, low_memory=False)
        # if 'id_usager' in df.columns:
        #     df = df.drop(columns=['id_usager'])
        result.append(df)
    combined_df = pd.concat(result, ignore_index=True)
    combined_df.to_csv(f'./Dataset/all_years/{file_name}{years[0]}-{years[-1]}.csv', index=False)

In [ ]:
caracteristiques = pd.read_csv('./Dataset/all_years/caracteristiques-2019-22.csv')

In [ ]:
# Remplacer les numéros des mois par leur noms
month_mapper = {
    1:'Janvier',
    2:'Février',
    3:'Mars',
    4:'Avril',
    5:'Mai',
    6:'Juin',
    7:'Juillet',
    8:'Août',
    9:'Septembre',
    10:'Octobre',
    11:'Novembre',
    12:'Décembre'
}

caracteristiques = replace_with_mapper(caracteristiques, 'mois', month_mapper)

caracteristiques.head(3)

In [ ]:
# Remplacer les heure par le moment de la journée
caracteristiques.hrmn = caracteristiques.hrmn.apply(replace_hour)

caracteristiques.head(3)

In [ ]:
departements = pd.read_csv('./Dataset/departements-france.csv')

In [ ]:
# Remplacer les numéros de départements par leur noms
departements_mapper = departements.set_index('code_departement')['nom_departement'].to_dict()
caracteristiques = replace_with_mapper(caracteristiques, 'dep', departements_mapper)

caracteristiques.head(3)

In [ ]:
usagers = pd.read_csv('./Dataset/all_years/usagers-2019-22.csv')

In [ ]:
usagers.head(3)

In [ ]:
usagers['an'] = usagers['Accident_Id'].map(caracteristiques.set_index('Accident_Id')['an'])
usagers['age_category'] = pd.cut(
    usagers['an'] - usagers['an_nais'],
    bins=[0, 18, 59, float('inf')],
    labels=["jeune", "adulte", "senior"],
    right=True
)

number_of_people = usagers.groupby(by='Accident_Id')['age_category'].value_counts().unstack().fillna(0)
number_of_people.reset_index(inplace=True)

caracteristiques = caracteristiques.merge(number_of_people, on='Accident_Id', how='right')

# Clean
usagers.drop(columns=['an'])
usagers.drop(columns=['age_category'])

In [ ]:
caracteristiques.head(15)